<a href="https://colab.research.google.com/github/victoralcantara75/CNN-FLOWER-CLASSIFY/blob/master/CNN_FLOWER_CLASSIFY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#CNN PARA CLASSIFICAÇÃO DE FLORES
## SIN493 - DEEP LEARNING PARA VISÃO COMPUTACIONAL

##PYTORCH E CUDA

In [0]:
#http://pytorch.org/

from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch

In [0]:
# verificando se o CUDA está disponível
GPUavailable = torch.cuda.is_available()

if GPUavailable:
    print('Treinamento em GPU!')
else:
    print('Treinamento em CPU!')

In [0]:
!pip uninstall -y Pillow
!pip install Pillow==5.3.0
import PIL
print(PIL.PILLOW_VERSION)

In [0]:
#IMPORTACOES NECESSARIAS
import os
import numpy as np
import torch
from torchvision import datasets, models
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler
import matplotlib.pyplot as plt
import torch.nn as nn 
import torch.optim as optim

##DATASET

In [0]:
#BAIXANDO DATASET 
!wget -cq https://github.com/udacity/pytorch_challenge/raw/master/cat_to_name.json
!wget -cq https://s3.amazonaws.com/content.udacity-data.com/courses/nd188/flower_data.zip
!unzip -qq flower_data.zip

In [0]:
#PEGANDO DIRETORIO DO DATASET DE TREINO E TESTE
diretorio_dados = 'flower_data'
diretorio_treino = diretorio_dados + '/train'
diretorio_validacao = diretorio_dados + '/valid'

In [0]:
# APLICANDO AS TRANSFORMACOES NECESSARIAS E O DATA AUGMENTATION

transformacao_treino = transforms.Compose([ 
                       transforms.RandomRotation(10),      #ROTAÇÃO DAS IMAGENS
                       transforms.RandomResizedCrop(224),  #INPUT SIZE 224X224
                       transforms.RandomHorizontalFlip(),  #ESPELHAMENTO
                       transforms.ToTensor(),
                       transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                ])
transformacao_validacao = transforms.Compose([ 
                          transforms.Resize(224),
                          transforms.RandomResizedCrop(224),
                          transforms.ToTensor(),
                          transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                ])


In [0]:
#PASSANDO OS DATASETS PARA OS VETORES (TREINO E TESTE)
dados_treino = datasets.ImageFolder(diretorio_treino, transformacao_treino)
dados_validacao = datasets.ImageFolder(diretorio_validacao, transformacao_validacao)

print('Numero de imagens de treino: ', len(dados_treino))
print('Numero de imagens de teste: ', len(dados_validacao))

In [0]:
# Data Loaders e Visualização

batch_size = 20
num_workers=4

#dataset_treino = datasets.ImageFolder(os.path.join(diretorio_dados), transformacao_treino)

# dataloaders
treino_loader = torch.utils.data.DataLoader(dados_treino, batch_size=batch_size, num_workers=num_workers, shuffle=True)
validacao_loader = torch.utils.data.DataLoader(dados_validacao, batch_size=batch_size,num_workers=num_workers, shuffle=True)

classes = dados_treino.classes
print(classes)

In [0]:
#MAPEANDO OS LABELS
import json

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

In [0]:
nomes_flores = [cat_to_name[classes[e]] for e in range (0, 101)]
print(nomes_flores)

In [0]:
# obtendo um batch de imagens de treinamento
dataiter = iter(treino_loader)
images, labels = dataiter.next()
images = images.numpy() # para exibir as imagens converter para numpy

# plotar as imagens no batch, com os rótulos correspondentes
fig = plt.figure(figsize=(25, 4))
for idx in np.arange(20):
    ax = fig.add_subplot(2, 20/2, idx+1, xticks=[], yticks=[])
    plt.imshow(np.transpose(images[idx], (1, 2, 0)))

##TREINO E CLASSIFICACAO VGG

In [0]:
#criando o modelo de vgg 16 pre treinada
VGG = models.vgg16(pretrained=True)
#visualizacao da arquitetura utilizada vgg16
print(VGG)

# movendo os tensors para GPU se o CUDA estiver disponível
if GPUavailable:
    VGG.cuda()


In [0]:
# Congelar o treinamento para todas as camadas de características
for param in VGG.features.parameters():
    param.requires_grad = False

In [0]:
n_inputs = VGG.classifier[6].in_features

# adicionar a última camada para 102 classes
# novas camadas automaticamente tem requires_grad = True
last_layer = nn.Linear(n_inputs, 102)

VGG.classifier[6] = last_layer

# verifique se sua última camada produz o número esperado de saídas
print(VGG.classifier[6].out_features)

In [0]:
# Especificando a loss function (categorical cross-entropy)
criterion = nn.CrossEntropyLoss()

# Definindo o Otimizador
optimizer = optim.SGD(VGG.parameters(), lr=0.01)

In [0]:
#TREINAMENTO 

epocas = 3

for epocas in range (1, epocas+1):

  treino_loss = 0.0
  
  VGG.train()
  for batch_i, (data, target) in enumerate(treino_loader):
        
        if GPUavailable:
            data, target = data.cuda(), target.cuda()
            
        optimizer.zero_grad()
        output = VGG(data)
        loss = criterion(output, target)  #batch loss
        loss.backward()
        optimizer.step()                  #otimizador SGD 
        treino_loss += loss.item()        #atualizacao do treinamento
        
        if batch_i % 20 == 19:
            print('Época %d, Batch %d loss: %.16f' % (epocas, batch_i + 1, treino_loss / 20))
            treino_loss = 0.0
        
 

In [0]:
#TESTE

test_loss = 0.0
class_correct = list(0. for i in range(102))
class_total = list(0. for i in range(102))

VGG.eval()

for data, target in validacao_loader:
    if GPUavailable:
        data, target = data.cuda(), target.cuda()
   
    output = VGG(data)
    loss = criterion(output, target)           #test loss
    test_loss += loss.item()*data.size(0)      #atualizando loss
    _, pred = torch.max(output, 1)    
    correct_tensor = pred.eq(target.data.view_as(pred))    #comparando respostas corretas
    correct = np.squeeze(correct_tensor.numpy()) if not GPUavailable else np.squeeze(correct_tensor.cpu().numpy())
   
    # calculando acurácia do teste
    for i in range(batch_size-2):
        label = target.data[i]
        class_correct[label] += correct[i].item()
        class_total[label] += 1
        

In [0]:
# calculando avg teste loss
test_loss = test_loss/len(validacao_loader.dataset)
print('Teste Loss: {:.6f}\n'.format(test_loss))

for i in range(102):
    if class_total[i] > 0:
        print('Acurácia do teste classe %5s: %2d%% (%2d/%2d)' % (str(i), 100 * class_correct[i] / class_total[i], np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Acurácia do teste classe %5s: N/A (sem exemplos de treinamento')

print('\nAcurácia Total: %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))


##TREINO E CLASSIFICAÇÃO RESNET

In [0]:
#criando o modelo de resnet pre treinada
resnet = models.resnet18(pretrained=True)
#visualizacao da arquitetura utilizada resnet
print(resnet)

# movendo os tensors para GPU se o CUDA estiver disponível
if GPUavailable:
    resnet.cuda()

In [0]:
# Especificando a loss function (categorical cross-entropy)
criterion = nn.CrossEntropyLoss()

# Definindo o Otimizador
optimizer = optim.SGD(resnet.parameters(), lr=0.01)

In [0]:
#TREINAMENTO 

epocas = 3

resnet.train()
for epocas in range (1, epocas+1):

  treino_loss = 0.0
  
 
  for batch_i, (data, target) in enumerate(treino_loader):
        
        if GPUavailable:
            data, target = data.cuda(), target.cuda()
            
        optimizer.zero_grad()
        output = resnet(data)
        loss = criterion(output, target)  #batch loss
        loss.backward()
        optimizer.step()                  #otimizador SGD 
        treino_loss += loss.item()        #atualizacao do treinamento
        
        if batch_i % 20 == 19:
            print('Época %d, Batch %d loss: %.16f' % (epocas, batch_i + 1, treino_loss / 20))
            treino_loss = 0.0

In [0]:
#TESTE

test_loss = 0.0
class_correct = list(0. for i in range(102))
class_total = list(0. for i in range(102))

resnet.eval()

for data, target in validacao_loader:
    if GPUavailable:
        data, target = data.cuda(), target.cuda()
   
    output = resnet(data)
    loss = criterion(output, target)           #test loss
    test_loss += loss.item()*data.size(0)      #atualizando loss
    _, pred = torch.max(output, 1)    
    correct_tensor = pred.eq(target.data.view_as(pred))    #comparando respostas corretas
    correct = np.squeeze(correct_tensor.numpy()) if not GPUavailable else np.squeeze(correct_tensor.cpu().numpy())
   
    # calculando acurácia do teste
    for i in range(batch_size-2):
        label = target.data[i]
        class_correct[label] += correct[i].item()
        class_total[label] += 1

In [0]:
# calculando avg teste loss
test_loss = test_loss/len(validacao_loader.dataset)
print('Teste Loss: {:.6f}\n'.format(test_loss))

for i in range(102):
    if class_total[i] > 0:
        print('Acurácia do teste classe %5s: %2d%% (%2d/%2d)' % (str(i), 100 * class_correct[i] / class_total[i],np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Acurácia do teste classe %5s: N/A (sem exemplos de treinamento')

print('\nAcurácia Total: %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))